In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install category_encoders
import category_encoders as ce

     |████████████████████████████████| 82 kB 441 kB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


##import libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
# import torch.utils.model_zoo as model_zoo
# from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import time
import os
from collections import defaultdict
from datetime import timedelta
from datetime import datetime
import pytz
import pickle
import seaborn as sns

from sklearn.model_selection import KFold
import random
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler

In [ ]:

!cp /content/drive/"My Drive"/e3.csv /content/
# !cp /content/drive/"My Drive"/e3-scaled-withclass.csv /content/


##read data

In [ ]:
csvdata = pd.read_csv('e3.csv')

In [ ]:
#delete irrelevant columns
del csvdata['attackType']
del csvdata['attackID']
del csvdata['attackDescription']
del csvdata['Flows']
del csvdata['Tos']
csvdata

,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Flags,class,Bytes
0,0.000,TCP,26913_25,43618,EXT_SERVER,23.0,1,....S.,suspicious,46
1,0.000,TCP,EXT_SERVER,23,26913_25,43618.0,1,.A.R..,suspicious,40
2,0.000,TCP,26914_116,1817,EXT_SERVER,23.0,1,....S.,suspicious,46
3,0.000,TCP,EXT_SERVER,23,26914_116,1817.0,1,.A.R..,suspicious,40
4,0.001,TCP,26915_202,57272,EXT_SERVER,1433.0,1,....S.,suspicious,46
...,...,...,...,...,...,...,...,...,...,...
153021,67601.294,TCP,OPENSTACK_NET,49493,EXT_SERVER,8082.0,160681,.APRS.,normal,12200000
153022,67601.294,TCP,EXT_SERVER,8082,OPENSTACK_NET,49494.0,172810,.AP.S.,normal,465700000
153023,67601.294,TCP,OPENSTACK_NET,49494,EXT_SERVER,8082.0,170412,.APRS.,normal,28500000
153024,67601.103,TCP,EXT_SERVER,8082,OPENSTACK_NET,49495.0,148533,.AP.S.,normal,394900000


##preprocessing

In [ ]:
#change object values of "Bytes" column to int values (for example: 1M -> 1000000)
multipliers = {'K':1000, 'M':1000000, 'B':1000000000}

def string_to_int(string):
  if string.find('M') == -1:
      return int(string)
  mult = multipliers[string[-1]] # look up suffix to get multiplier
  # convert number to float, multiply by multiplier, then make int
  return int(float(string[:-1]) * mult)

bytes_new = list(map(string_to_int, csvdata['Bytes'].astype('str') ))
print("len of bytes_new: ", len(bytes_new))

del csvdata['Bytes']

csvdata['Bytes'] = bytes_new
csvdata

len of bytes_new:  153026


,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Flags,class,Bytes
0,0.000,TCP,26913_25,43618,EXT_SERVER,23.0,1,....S.,suspicious,46
1,0.000,TCP,EXT_SERVER,23,26913_25,43618.0,1,.A.R..,suspicious,40
2,0.000,TCP,26914_116,1817,EXT_SERVER,23.0,1,....S.,suspicious,46
3,0.000,TCP,EXT_SERVER,23,26914_116,1817.0,1,.A.R..,suspicious,40
4,0.001,TCP,26915_202,57272,EXT_SERVER,1433.0,1,....S.,suspicious,46
...,...,...,...,...,...,...,...,...,...,...
153021,67601.294,TCP,OPENSTACK_NET,49493,EXT_SERVER,8082.0,160681,.APRS.,normal,12200000
153022,67601.294,TCP,EXT_SERVER,8082,OPENSTACK_NET,49494.0,172810,.AP.S.,normal,465700000
153023,67601.294,TCP,OPENSTACK_NET,49494,EXT_SERVER,8082.0,170412,.APRS.,normal,28500000
153024,67601.103,TCP,EXT_SERVER,8082,OPENSTACK_NET,49495.0,148533,.AP.S.,normal,394900000


In [ ]:
#convert categorical values to int values

csvdata = pd.get_dummies(csvdata, columns=['Proto'])

csvdata = pd.get_dummies(csvdata, columns=['Flags'])

encoder = ce.BinaryEncoder(cols=['Src IP Addr'])
csvdata = encoder.fit_transform(csvdata)
csvdata

encoder = ce.BinaryEncoder(cols=['Dst IP Addr'])
csvdata = encoder.fit_transform(csvdata)
csvdata

encoder = ce.BinaryEncoder(cols=['Duration'])
csvdata = encoder.fit_transform(csvdata)
csvdata

encoder = ce.BinaryEncoder(cols=['Src Pt'])
csvdata = encoder.fit_transform(csvdata)
csvdata

encoder = ce.BinaryEncoder(cols=['Dst Pt'])
csvdata = encoder.fit_transform(csvdata)
csvdata


,Duration_0,Duration_1,Duration_2,Duration_3,Duration_4,Duration_5,Duration_6,Duration_7,Duration_8,Duration_9,Duration_10,Duration_11,Duration_12,Duration_13,Duration_14,Src IP Addr_0,Src IP Addr_1,Src IP Addr_2,Src IP Addr_3,Src IP Addr_4,Src IP Addr_5,Src IP Addr_6,Src IP Addr_7,Src IP Addr_8,Src IP Addr_9,Src IP Addr_10,Src IP Addr_11,Src IP Addr_12,Src IP Addr_13,Src Pt_0,Src Pt_1,Src Pt_2,Src Pt_3,Src Pt_4,Src Pt_5,Src Pt_6,Src Pt_7,Src Pt_8,Src Pt_9,Src Pt_10,...,Dst Pt_6,Dst Pt_7,Dst Pt_8,Dst Pt_9,Dst Pt_10,Dst Pt_11,Dst Pt_12,Dst Pt_13,Dst Pt_14,Dst Pt_15,Packets,class,Bytes,Proto_GRE,Proto_ICMP,Proto_TCP,Proto_UDP,Flags_ 0x52,Flags_ 0x5b,Flags_ 0xc2,Flags_ 0xd3,Flags_ 0xd6,Flags_ 0xd7,Flags_ 0xdb,Flags_ 0xdf,Flags_......,Flags_....S.,Flags_...R..,Flags_...RS.,Flags_.A....,Flags_.A..S.,Flags_.A..SF,Flags_.A.R..,Flags_.A.R.F,Flags_.A.RS.,Flags_.A.RSF,Flags_.AP.S.,Flags_.AP.SF,Flags_.APRS.,Flags_.APRSF
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,1,suspicious,46,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,1,suspicious,40,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,1,suspicious,46,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,suspicious,40,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,1,suspicious,46,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153021,1,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,1,1,0,1,0,1,0,1,...,1,1,1,0,1,1,1,0,1,0,160681,normal,12200000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
153022,1,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,1,0,1,1,0,1,172810,normal,465700000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
153023,1,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,...,1,1,1,0,1,1,1,0,1,0,170412,normal,28500000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
153024,1,0,0,0,0,0,0,1,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1,148533,normal,394900000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
#min-max scaling

for column in csvdata:
  if column.find('_') != -1:
    continue
  if column.find('class') != -1:
    continue
  print('-----')
  print(column)
  min = csvdata[column].min()
  max = csvdata[column].max()
  print(min)
  print(max)
  if min != 0:
    csvdata[column] = csvdata[column] - min
  if max != min:
    csvdata[column] = csvdata[column] / (max - min)

csvdata

-----
Packets
1
176609
-----
Bytes
28
509200000


,Duration_0,Duration_1,Duration_2,Duration_3,Duration_4,Duration_5,Duration_6,Duration_7,Duration_8,Duration_9,Duration_10,Duration_11,Duration_12,Duration_13,Duration_14,Src IP Addr_0,Src IP Addr_1,Src IP Addr_2,Src IP Addr_3,Src IP Addr_4,Src IP Addr_5,Src IP Addr_6,Src IP Addr_7,Src IP Addr_8,Src IP Addr_9,Src IP Addr_10,Src IP Addr_11,Src IP Addr_12,Src IP Addr_13,Src Pt_0,Src Pt_1,Src Pt_2,Src Pt_3,Src Pt_4,Src Pt_5,Src Pt_6,Src Pt_7,Src Pt_8,Src Pt_9,Src Pt_10,...,Dst Pt_6,Dst Pt_7,Dst Pt_8,Dst Pt_9,Dst Pt_10,Dst Pt_11,Dst Pt_12,Dst Pt_13,Dst Pt_14,Dst Pt_15,Packets,class,Bytes,Proto_GRE,Proto_ICMP,Proto_TCP,Proto_UDP,Flags_ 0x52,Flags_ 0x5b,Flags_ 0xc2,Flags_ 0xd3,Flags_ 0xd6,Flags_ 0xd7,Flags_ 0xdb,Flags_ 0xdf,Flags_......,Flags_....S.,Flags_...R..,Flags_...RS.,Flags_.A....,Flags_.A..S.,Flags_.A..SF,Flags_.A.R..,Flags_.A.R.F,Flags_.A.RS.,Flags_.A.RSF,Flags_.AP.S.,Flags_.AP.SF,Flags_.APRS.,Flags_.APRSF
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0.000000,suspicious,3.534957e-08,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0.000000,suspicious,2.356638e-08,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0.000000,suspicious,3.534957e-08,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,0.000000,suspicious,2.356638e-08,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0.000000,suspicious,3.534957e-08,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153021,1,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,1,1,0,1,0,1,0,1,...,1,1,1,0,1,1,1,0,1,0,0.909812,normal,2.395910e-02,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
153022,1,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,1,0,1,1,0,1,0.978489,normal,9.145719e-01,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
153023,1,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,...,1,1,1,0,1,1,1,0,1,0,0.964911,normal,5.597010e-02,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
153024,1,0,0,0,0,0,0,1,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1,0.841026,normal,7.755302e-01,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
for column in csvdata:
  print(csvdata[column].value_counts())
  print()
  print()

0    152575
1       451
Name: Duration_0, dtype: int64


0    99037
1    53989
Name: Duration_1, dtype: int64


0    111863
1     41163
Name: Duration_2, dtype: int64


0    111358
1     41668
Name: Duration_3, dtype: int64


0    97378
1    55648
Name: Duration_4, dtype: int64


0    84536
1    68490
Name: Duration_5, dtype: int64


1    76962
0    76064
Name: Duration_6, dtype: int64


0    79714
1    73312
Name: Duration_7, dtype: int64


0    88779
1    64247
Name: Duration_8, dtype: int64


0    100231
1     52795
Name: Duration_9, dtype: int64


0    101291
1     51735
Name: Duration_10, dtype: int64


0    84255
1    68771
Name: Duration_11, dtype: int64


0    89880
1    63146
Name: Duration_12, dtype: int64


0    94904
1    58122
Name: Duration_13, dtype: int64


1    86619
0    66407
Name: Duration_14, dtype: int64


0    152757
1       269
Name: Src IP Addr_0, dtype: int64


0    114802
1     38224
Name: Src IP Addr_1, dtype: int64


0    119837
1     33189
Name: Src IP Add

In [ ]:
csvdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153026 entries, 0 to 153025
Columns: 105 entries, Duration_0 to Flags_.APRSF
dtypes: float64(2), int64(75), object(1), uint8(27)
memory usage: 95.0+ MB


In [ ]:

normal = csvdata.query('`class`=="normal"')
del normal['class']

attacker = csvdata.query('`class`=="attacker"')
del attacker['class']

victim = csvdata.query('`class`=="victim"')
del victim['class']

unknown = csvdata.query('`class`=="unknown"')
del unknown['class']

suspicious = csvdata.query('`class`=="suspicious"')
del suspicious['class']

print("Normal len: ", len(normal))
print("Victim len: ", len(victim))
print("Attacker len: ", len(attacker))
print("Unknown len: ", len(unknown))
print("Suspicious len: ", len(suspicious))

Normal len:  6180
Victim len:  5902
Attacker len:  9255
Unknown len:  33837
Suspicious len:  97852


##Autoencoder network

In [ ]:
class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.e1 = nn.Linear(in_features=kwargs["input_shape"], out_features=165 ) # 1st Hidden Layer    
        self.output_layer = nn.Linear(in_features=165, out_features=kwargs["input_shape"])

    def forward(self, features):
        out = torch.relu(self.e1(features))
        out = self.output_layer(out)
        out = torch.sigmoid(out)
        return out


##Data loader functions

In [ ]:
class Loader(torch.utils.data.Dataset):
    def __init__(self, data):
        super(Loader, self).__init__()
        self.dataset = data
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        row = self.dataset.iloc[idx]
        data = torch.from_numpy(np.array(row)).float()
        return data

class Data_Loader(Loader):
    def __init__(self, csvfile):
        super(Data_Loader, self).__init__()
        self.dataset = pd.read_csv(
                       csvfile,
                       index_col=False
                       )        

##Training and testing functions

In [ ]:
def train_epoch(model,device,dataloader,loss_fn,optimizer):
    train_loss = 0.0
    model.train()

    for data in dataloader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, data)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    return train_loss

In [ ]:
def valid_epoch(model,device,dataloader,loss_fn):
    valid_loss = 0.0
    model.eval()
    for data in dataloader:

        data = data.to(device)
        output = model(data)
        loss=loss_fn(output, data)
        valid_loss += loss.item()

    return valid_loss

In [ ]:
batch_size = 1
num_epochs = 30
size = csvdata.shape[1] - 1
criterion = nn.MSELoss()
k=10
splits=KFold(n_splits=k,shuffle=True,random_state=42)

normal_foldperf={}
victim_foldperf={}
attacker_foldperf={}
unknown_foldperf={}
suspicious_foldperf={}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

normal_model = AE(input_shape=size)
attacker_model = AE(input_shape=size)
victim_model = AE(input_shape=size)
unknown_model = AE(input_shape=size)
suspicious_model = AE(input_shape=size)

normal_model.to(device)
attacker_model.to(device)
victim_model.to(device)
unknown_model.to(device)
suspicious_model.to(device)

normal_optimizer = optim.RMSprop(normal_model.parameters(), lr=0.0001385450437405928)
attacker_optimizer = optim.RMSprop(attacker_model.parameters(), lr=0.0001385450437405928)
victim_optimizer = optim.RMSprop(victim_model.parameters(), lr=0.0001385450437405928)
unknown_optimizer = optim.RMSprop(unknown_model.parameters(), lr=0.0001385450437405928)
suspicious_optimizer = optim.RMSprop(suspicious_model.parameters(), lr=0.0001385450437405928)

normal_metrics = defaultdict(list)
attacker_metrics = defaultdict(list)
victim_metrics = defaultdict(list)
unknown_metrics = defaultdict(list)
suspicious_metrics = defaultdict(list)

# normal_history = {'train_loss': [], 'test_loss': []}
# attacker_history = {'train_loss': [], 'test_loss': []}
# victim_history = {'train_loss': [], 'test_loss': []}
# unknown_history = {'train_loss': [], 'test_loss': []}
# suspicious_history = {'train_loss': [], 'test_loss': []}

save_dir = '/content/ckpts/'
if not os.path.exists(save_dir):
  os.makedirs(save_dir)


In [ ]:
def trainClassifier(className, data, model, optimizer, metrics, foldperf):
  for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(data)))):

      print('Fold {}'.format(fold))

      train_sampler = SubsetRandomSampler(train_idx)
    
      test_sampler = SubsetRandomSampler(val_idx)

      dataset = Loader(data)
      train_loader = torch.utils.data.DataLoader(
              dataset,
              batch_size=batch_size,
              sampler=train_sampler
          )
      test_loader = torch.utils.data.DataLoader(
              dataset,
              batch_size=batch_size,
              sampler=test_sampler
          )
      

      history = {'train_loss': [], 'test_loss': []}

      for epoch in range(num_epochs):

          train_loss = train_epoch(model,device,train_loader,criterion,optimizer)
          test_loss = valid_epoch(model,device,test_loader,criterion)

          train_loss = train_loss / len(train_loader.sampler)
          test_loss = test_loss / len(test_loader.sampler)

          print("Epoch:{}/{} AVG Training Loss:{} AVG Test Loss:{} ".format(epoch + 1, num_epochs, train_loss, test_loss))

          metrics['train_loss'].append(train_loss)
          metrics['val_loss'].append(test_loss)

          history['train_loss'].append(train_loss)
          history['test_loss'].append(test_loss)
      

          # !rsync -a /content/ckpts/ /content/drive/"My Drive"/AE/e3-kfold-new/


      foldperf['fold{}'.format(fold)] = history
      torch.save(model.state_dict(), os.path.join(save_dir, '{}_cross_fold_{}.pth'.format(className, fold))) 

  # torch.save(model,'cross_victim.pt')
  torch.save(model.state_dict(), os.path.join(save_dir, '{}_cross.pth'.format(className)))
  !rsync -a /content/ckpts/ /content/drive/"My Drive"/AE/e3-kfold-new/

In [40]:
trainClassifier("Normal", normal, normal_model, normal_optimizer, normal_metrics, normal_foldperf)

Fold 0
Epoch:1/30 AVG Training Loss:0.05946764146359232 AVG Test Loss:0.037974203972923525 
Epoch:2/30 AVG Training Loss:0.025248133374966817 AVG Test Loss:0.015290094719608962 
Epoch:3/30 AVG Training Loss:0.008866374185228442 AVG Test Loss:0.004546268076996069 
Epoch:4/30 AVG Training Loss:0.002443208238407787 AVG Test Loss:0.0012789769651512179 
Epoch:5/30 AVG Training Loss:0.0007394701295322521 AVG Test Loss:0.0005339041892223438 
Epoch:6/30 AVG Training Loss:0.0003135661436167167 AVG Test Loss:0.00028064447195693196 
Epoch:7/30 AVG Training Loss:0.00016801097568641218 AVG Test Loss:0.0001853000364895459 
Epoch:8/30 AVG Training Loss:0.00010393807257271392 AVG Test Loss:0.00013192692875053902 
Epoch:9/30 AVG Training Loss:7.104839396749602e-05 AVG Test Loss:0.00010151655649305209 
Epoch:10/30 AVG Training Loss:5.138865845865632e-05 AVG Test Loss:8.794583272850437e-05 
Epoch:11/30 AVG Training Loss:3.9597083569496694e-05 AVG Test Loss:7.327248004684764e-05 
Epoch:12/30 AVG Training 

In [ ]:
trainClassifier("Attacker", attacker, attacker_model, attacker_optimizer, attacker_metrics, attacker_foldperf)

Fold 0
Epoch:1/30 AVG Training Loss:0.04643849442611321 AVG Test Loss:0.02155913429014932 
Epoch:2/30 AVG Training Loss:0.012666557332132266 AVG Test Loss:0.006599255099218262 
Epoch:3/30 AVG Training Loss:0.003998324631469696 AVG Test Loss:0.0021356690241872874 
Epoch:4/30 AVG Training Loss:0.0013678114856429569 AVG Test Loss:0.0008057103244775948 
Epoch:5/30 AVG Training Loss:0.0005336312335075279 AVG Test Loss:0.0003681342913036954 
Epoch:6/30 AVG Training Loss:0.0002514949207696424 AVG Test Loss:0.00018266834866902077 
Epoch:7/30 AVG Training Loss:0.00014073895414731336 AVG Test Loss:0.00011657261599994465 
Epoch:8/30 AVG Training Loss:8.577525386394202e-05 AVG Test Loss:7.82876346964966e-05 
Epoch:9/30 AVG Training Loss:5.780152061300683e-05 AVG Test Loss:5.0768019988162844e-05 
Epoch:10/30 AVG Training Loss:4.21176090992741e-05 AVG Test Loss:4.1342823981791975e-05 
Epoch:11/30 AVG Training Loss:3.496009055835434e-05 AVG Test Loss:3.9039024232311636e-05 
Epoch:12/30 AVG Training L

In [ ]:
trainClassifier("Victim", victim, victim_model, victim_optimizer, victim_metrics, victim_foldperf)

Fold 0
Epoch:1/30 AVG Training Loss:0.060170489276044396 AVG Test Loss:0.03663204667393225 
Epoch:2/30 AVG Training Loss:0.025880786302429154 AVG Test Loss:0.01674265445993233 
Epoch:3/30 AVG Training Loss:0.01083417883767057 AVG Test Loss:0.006708156280834134 
Epoch:4/30 AVG Training Loss:0.004373089040651179 AVG Test Loss:0.0028291578013008674 
Epoch:5/30 AVG Training Loss:0.0019025800053006118 AVG Test Loss:0.0012540731837060192 
Epoch:6/30 AVG Training Loss:0.0008968813973951067 AVG Test Loss:0.0005747772811077001 
Epoch:7/30 AVG Training Loss:0.0004640780551565798 AVG Test Loss:0.0002604128544351381 
Epoch:8/30 AVG Training Loss:0.0002726447299827811 AVG Test Loss:0.0001381766827610124 
Epoch:9/30 AVG Training Loss:0.0001750795219033089 AVG Test Loss:9.04699477529107e-05 
Epoch:10/30 AVG Training Loss:0.00011290374947047033 AVG Test Loss:5.0310474798007e-05 
Epoch:11/30 AVG Training Loss:8.005392332696273e-05 AVG Test Loss:3.220296574127918e-05 
Epoch:12/30 AVG Training Loss:5.823

##Find Best model of cross validation folds

In [ ]:
def findBestModelIndex(className, foldperf, k): # k=number of folds
  loss = []
  for f in range(0,k):
    loss.append(sum(foldperf['fold{}'.format(f)]['train_loss']) + sum(foldperf['fold{}'.format(f)]['test_loss']))
  
  min_value = np.min(loss)
  min_index = loss.index(min_value)
  print(className, "best model index:", min_index,"and value:", min_value)

In [41]:
findBestModelIndex("Normal", normal_foldperf, k)

Normal best model index: 9 and value: 6.1163509167782e-05


In [42]:
!cp /content/ckpts/Normal_cross_fold_9.pth /content/drive/"My Drive"/AE/e3-kfold-new/best_models/Normal.pth

In [ ]:
!cp /content/ckpts/Normla_cross_fold_3.pth /content/drive/"My Drive"/AE/e3-kfold-new/best_models/Attacker.pth

In [ ]:
findBestModelIndex("Attacker", attacker_foldperf, k)

Attacker best model index: 3 and value: 0.00041561010339621185


In [ ]:
!cp /content/ckpts/Attacker_cross_fold_3.pth /content/drive/"My Drive"/AE/e3-kfold-new/best_models/Attacker.pth

In [ ]:
findBestModelIndex("Victim", victim_foldperf, k)

Victim best model index: 5 and value: 0.0005973509285028913


In [ ]:
!cp /content/ckpts/Victim_cross_fold_5.pth /content/drive/"My Drive"/AE/e3-kfold-new/best_models/Victim.pth

##Training performance

In [49]:
def trainingPerfprmance(className, foldperf, k):
  testl_f,tl_f = [],[]

  for f in range(0, k):
      tl_f.append(np.mean(foldperf['fold{}'.format(f)]['train_loss']))
      testl_f.append(np.mean(foldperf['fold{}'.format(f)]['test_loss']))

  print('Performance of', className, 'in {} fold cross validation'.format(k))
  print("Average Training Loss: {} \t Average Test Loss: {}".format(np.mean(tl_f),np.mean(testl_f)))
  print()


In [50]:
trainingPerfprmance("Normal", normal_foldperf, k)

Performance of Normal in 10 fold cross validation
Average Training Loss: 0.0003274487429321198 	 Average Test Loss: 0.00020558926182749748



In [51]:
trainingPerfprmance("Attacker", attacker_foldperf, k)

Performance of Attacker in 10 fold cross validation
Average Training Loss: 0.00023103662095637673 	 Average Test Loss: 0.00011846412644365276



##Loss chart

In [ ]:
_, ax = plt.subplots(1,1,figsize=(15,10))
ax.set_title('Loss')
ax.plot(normal_metrics['train_loss'])
ax.plot(normal_metrics['val_loss'])

In [ ]:
_, ax = plt.subplots(1,1,figsize=(15,10))
ax.set_title('Loss')
ax.plot(attacker_metrics['train_loss'])
ax.plot(attacker_metrics['val_loss'])

In [ ]:
_, ax = plt.subplots(1,1,figsize=(15,10))
ax.set_title('Loss')
ax.plot(victim_metrics['train_loss'])
ax.plot(victim_metrics['val_loss'])